# Homework 2 from Martin Gräf, Richard Baumann and Thomas Block

After taking a look at the one-dimensional ising model in Homework 1, we now take a look at the two-dimensional variant. Again we have the hamiltonian:


\begin{equation}
    H(\sigma) = - J \sum_{\langle x~y\rangle} \sigma_x \sigma_y - h \sum_x \sigma_x
\end{equation}





In [8]:
#just importing stuff, nothing to see here yet :)

import numpy as np 
import random as rd
import matplotlib.pyplot as plt
import math
import itertools 

This time we not only look at the particles left and right of the observed particle, but at the particles above and below. Therefore the hamiltonian looks like this:

\begin{equation}
    H(\sigma) = - J (\sigma_{x-1, y} \sigma_{x, y} + \sigma_{x+1, y} \sigma_{x1, y} + \sigma_{x, y-1} \sigma_{x, y} + \sigma_{x+1, y} \sigma_{x, y}) - h \sigma_{x, y}
\end{equation}

As we can see in the equation above, this hamiltonian looks at all adjacent spins. Therefore, the implementation looks something like that:


In [ ]:
#Spin is supposed to be a NxN big array. Position has to be an array of the form [x, y] to indicate the position of the spin.
def hamiltionian(spin, position, j, h):
    minuend = 0
    for i in [-1, 1]
        #Todo periocicallyity
        #Left/right spin
        minuend = minuend + spin[position[0]+i][position[1]]*spin[position[0]][position[1]]
        #Lower/upper spin
        minuend = minuend + spin[position[0]][position[1]+i]*spin[position[0]][position[1]] 
    return (-j*minuend-h*spin[position[0]][position[1]])

Now we have to generate a spin array, that can be used for the Hamiltonian. This spin-arry is $ N_{x} $ x $ N_{y} $ large, however in accordance to the exercise sheet we choose $ N_{x}=N_{y} $.

In [44]:
def gen_spin_array(length):
    configurations=[]
    current_configuration=[]
    for j in range(length):
        for i in range(length):
            #a random integer between 1 and 3 (1, 2)
            current_configuration=current_configuration+[2*rd.randint(1, 2)-3]
        configurations=configurations+[current_configuration]
        current_configuration=[]
    return configurations

[[-1, -1, 1, 1, 1], [1, 1, -1, 1, 1], [1, 1, -1, 1, -1], [1, -1, 1, 1, -1], [1, 1, -1, 1, -1]]


In [ ]:
Here comes the part that is different to the last homework. Instead of brute forcing the solution we now have a plan.